In [1]:
import torch

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [2]:
import os
print(os.cpu_count())

16


In [1]:
from datasets import load_dataset
from torch.utils.data import DataLoader

dataset = load_dataset("doof-ferb/vlsp2020_vinai_100h", split="train", streaming=True)
dataset.set_format(type="torch", columns=["audio", "transcription"])
dataloader = DataLoader(dataset, batch_size=4)

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

AttributeError: 'IterableDataset' object has no attribute 'set_format'